In [1]:
%load_ext autoreload
%autoreload 2

#%load_ext memory_profiler

from vflow import Vset
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from functools import partial

from numpy.testing import assert_equal

In [2]:
def make_train_test():
    X, y = make_classification(n_samples=1000, n_features=5)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    return(X_train, X_test, y_train, y_test)

train_test_vset = Vset(name = 'train_test', vfuncs=[make_train_test for i in range(10)])
train_test_vset_lazy = Vset(name = 'train_test', vfuncs=[make_train_test for i in range(10)], lazy=True)

# non-lazy eval
X_train, _, _, _ = train_test_vset(n_out=4, keys=['X_train', 'X_test', 'y_train', 'y_test'])

X_trains = list(X_train.values())
print(len(X_trains)) # 10 datasets + 1 for __prev__
print(X_trains[0].shape)
print(X_trains[0])

11
(750, 5)
[[ 1.19918727 -0.86090001  0.09363988 -1.37406474  0.49181207]
 [ 1.1297936   0.39286664  0.56225487 -0.74515435  1.31628301]
 [-1.69019637  0.2607743  -0.33707541  1.50196813 -1.36806542]
 ...
 [-1.15485274  0.83164691  1.20478409  1.32443985 -0.47180545]
 [-1.42322518  0.38186067  0.72810122  1.33877958 -1.0370119 ]
 [ 1.33715518 -0.69005709 -1.09000153 -1.40899358  0.73959803]]


In [3]:
# lazy eval
X_train, _, y_train, _ = train_test_vset_lazy(n_out=4, keys=['X_train', 'X_test', 'y_train', 'y_test'])

X_trains = list(X_train.values())
print(len(X_trains)) # 10 promises + 1 for __prev__
promise_X = X_trains[0]
print(promise_X) # the values are (unfulfilled) promises to call sep_dicts since n_out > 1

inner_promise_X = promise_X.args[0]
print(inner_promise_X.vfunc.vfunc) # the arg passed to sep_dicts is itself a promise to call make_train_test

11
Unfulfilled VfuncPromise(func=<function sep_dicts.<locals>.<lambda> at 0x7f9b1024d940>, args=(Unfulfilled VfuncPromise(func=<vflow.vfunc.Vfunc object at 0x7f9ad24dbd30>, args=()), 0))
<function make_train_test at 0x7f9ad24cb550>


In [4]:
y_trains = list(y_train.values()) # y_train is also a dict of unfulfilled promises
promise_y = y_trains[0]
inner_promise_y = promise_y.args[0]
print(promise_y)

Unfulfilled VfuncPromise(func=<function sep_dicts.<locals>.<lambda> at 0x7f9ad24cba60>, args=(Unfulfilled VfuncPromise(func=<vflow.vfunc.Vfunc object at 0x7f9ad24dbd30>, args=()), 2))


In [5]:
# calling the promise fulfills it and caches the value for future calls
promise_X()
print(f'promise_X called: {promise_X.called}')
print(f'promise_X call cached: {id(promise_X.value) == id(promise_X())}')
print(promise_X)

promise_X called: True
promise_X call cached: True
Fulfilled VfuncPromise([[-0.4986064   0.25019779 -1.18996883 -0.6757745  -0.06555213]
 [ 0.35903005  1.00652012 -0.34385297  1.23666652 -1.21683085]
 [ 0.31831027 -0.54928398 -0.20093244  1.03149028 -0.96942001]
 ...
 [ 0.11033152  1.10606787 -1.20645313  1.06767533 -1.53277576]
 [ 0.16540139  1.19916305 -0.39988871  0.72056736 -0.8147955 ]
 [-0.82251335 -0.77172041 -0.51591735 -2.01874352  1.41526363]])


In [6]:
# the inner promises are shared by corresponding values of X_train and y_train
print(f'promise_y called: {inner_promise_y.called}')

print(promise_y) # promise_y's first arg is now a fulfilled promise (fulfilled by the call promise_X())

print(f'promise_y call cached: {id(inner_promise_y.value) == id(inner_promise_y())}')

promise_y called: True
Unfulfilled VfuncPromise(func=<function sep_dicts.<locals>.<lambda> at 0x7f9ad24cba60>, args=(Fulfilled VfuncPromise((array([[-0.4986064 ,  0.25019779, -1.18996883, -0.6757745 , -0.06555213],
       [ 0.35903005,  1.00652012, -0.34385297,  1.23666652, -1.21683085],
       [ 0.31831027, -0.54928398, -0.20093244,  1.03149028, -0.96942001],
       ...,
       [ 0.11033152,  1.10606787, -1.20645313,  1.06767533, -1.53277576],
       [ 0.16540139,  1.19916305, -0.39988871,  0.72056736, -0.8147955 ],
       [-0.82251335, -0.77172041, -0.51591735, -2.01874352,  1.41526363]]), array([[ 0.46852662, -1.6446843 , -0.03677856,  1.35649023, -1.15427143],
       [-0.05815303, -0.03730004,  0.1759789 , -0.27544557,  0.32372088],
       [ 0.35612484,  0.53666212,  0.04372115,  0.98628672, -0.8018998 ],
       ...,
       [ 0.35271132,  0.62883962, -0.50705193,  1.32062987, -1.37359174],
       [-0.41612969,  0.62301024, -1.17465256, -0.45059802, -0.24580364],
       [ 0.22954556